<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN05/5.1-Einführung_CNN_PyTorch_CIFAR-10_ger.ipynb)

Der Aufbau eines neuronalen Faltungsnetzwerks (Convolutional Neural Network, CNN) mit PyTorch umfasst mehrere Schritte, darunter die Definition der Architektur des Netzwerks, die Vorbereitung der Daten, das Training des Modells und die Bewertung seiner Leistung. In diesem Artikel werden wir sehen, wie wir ein CNN-Netzwerk in PyTorch aufbauen können.

Inhaltsübersicht

Was sind Faltungsneuronale Netze?
Code-Implementierung für den Aufbau eines neuronalen Faltungsnetzwerks in PyTorch
* Schritt 1: Importieren der erforderlichen Bibliotheken
* Schritt 2: Vorbereiten des Datensatzes
* Schritt 3: Definieren der CNN-Architektur
* Schritt 4: Definieren der Verlustfunktion und des Optimierers
* Schritt 5: Trainieren des Netzwerks
* Schritt 6: Testen des Netzwerks

Vollständiger Code zum Aufbau eines CNN mit PyTorch



## Was sind Faltungsneuronale Netze?

Faltungsneuronale Netze (Convolutional Neural Networks, CNN) sind eine Klasse von tiefen neuronalen Netzen, die hauptsächlich für die Analyse visueller Bilder verwendet werden. Sie bestehen aus mehreren Schichten, darunter Faltungsschichten, Pooling-Schichten und vollständig verbundene Schichten. CNNs sind darauf ausgelegt, automatisch und adaptiv räumliche Hierarchien von Merkmalen aus Eingabebildern zu lernen. Hier sind einige Komponenten von CNN.

- *Faltungsschichten*: Diese Schichten wenden Faltungsoperationen auf die Eingabe an und extrahieren effektiv Merkmale wie Kanten, Texturen und Muster.
- *Pooling-Schichten*: Pooling-Schichten reduzieren die räumlichen Dimensionen des Inputs durch Downsampling und helfen so, die Rechenkomplexität zu verringern und die Überanpassung zu kontrollieren.
- *Aktivierungsfunktionen*: Aktivierungsfunktionen wie ReLU (Rectified Linear Unit) führen Nichtlinearität in das Netz ein, so dass es komplexe Beziehungen lernen kann.
- *Vollständig verbundene Schichten*: Diese auch als dichte Schichten bezeichneten Schichten führen eine Klassifizierung auf der Grundlage der von den vorherigen Schichten extrahierten Merkmale durch.
- *Verlustfunktion*: Die Verlustfunktion misst die Differenz zwischen der vorhergesagten Ausgabe des Netzes und der Grundwahrheit und steuert den Lernprozess des Netzes.
- *Optimierungsalgorithmus*: Algorithmen wie Stochastic Gradient Descent (SGD), Adam oder RMSprop werden verwendet, um die Parameter des Netzes durch Minimierung der Verlustfunktion zu optimieren.



## Implementierung des Aufbaus eines neuronalen Faltungsnetzes in PyTorch

### Schritt 1: Importieren der notwendigen Bibliotheken
In diesem Python-Codeblock importieren wir wichtige Module aus der PyTorch-Bibliothek, einem beliebten Open-Source-Framework für maschinelles Lernen.

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
# to tain on the GPU, we use CUDA 12.4
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


# Select device: CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Schritt 2: Vorbereiten des Datensatzes

- Dieser Code bereitet den CIFAR-10-Datensatz für das Training und Testen eines neuronalen Netzes mit PyTorch vor.
- Er definiert eine Reihe von Bildtransformationen, einschließlich der Umwandlung von Bildern in PyTorch-Tensoren und deren Normalisierung. Anschließend werden Datensatzobjekte für die Trainings- und Testdatensätze von CIFAR-10 erstellt, wobei das Stammverzeichnis, die Angabe, dass es sich um einen Trainings- oder Testdatensatz handelt, und die Transformationssequenz angegeben werden.
- Als Nächstes werden Datenlader für beide Datensätze erstellt, die das Laden der Daten in Stapeln, das Mischen der Daten und die Verwendung mehrerer Prozesse für ein schnelleres Laden der Daten ermöglichen.
- Schließlich werden die Klassenbezeichnungen für CIFAR-10 definiert, die die 10 verschiedenen Objektklassen im Datensatz repräsentieren. 
- Insgesamt bereitet dieser Code den CIFAR-10-Datensatz für die Verwendung beim Training und bei der Evaluierung neuronaler Netzmodelle vor.




In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def imshow(img):
    """Helper function to unnormalize and return an image for plotting."""
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    return np.transpose(npimg, (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)


def plot_batch(dataloader, classes, batch_size=4):
    """
    Plots a batch of images from the given dataloader in a single figure.

    Args:
        dataloader: PyTorch dataloader for CIFAR-10 dataset.
        classes: Tuple of class names in CIFAR-10.
        batch_size: Number of images to display in the batch.
    """
    # Get one batch of images and labels
    data_iter = iter(dataloader)
    images, labels = next(data_iter)

    # Create a figure with subplots
    fig, axes = plt.subplots(1, batch_size, figsize=(batch_size * 3, 3))
    for i in range(batch_size):
        img = imshow(images[i])  # Unnormalize and prepare the image
        axes[i].imshow(img)
        axes[i].set_title(classes[labels[i]])
        axes[i].axis("off")  # Hide axes

    plt.tight_layout()  # Adjust layout to avoid overlapping
    plt.show()


# Example usage with the CIFAR-10 trainloader
plot_batch(trainloader, classes, batch_size=4)


### Schritt 3: Definieren der CNN-Architektur
- Dieser Code definiert eine neuronale Netzwerkarchitektur unter Verwendung der `nn.Module`-Klasse von PyTorch. Die Klasse Net erbt von nn.Module und definiert die Schichten des Netzwerks in ihrer `__init__`-Methode.
- Es hat zwei Faltungsschichten (conv1 und conv2) mit ReLU-Aktivierungsfunktionen, gefolgt von Max-Pooling-Schichten (`pool`). Die vollständig verbundenen Schichten (`fc1`, `fc2` und `fc3`) verarbeiten die Ausgabe der Faltungsschichten.
- Die Vorwärtsmethode `forward` definiert den Vorwärtsdurchlauf des Netzes, bei dem die Eingabe `x` nacheinander durch jede Schicht geleitet wird. Die view-Methode formt die Ausgabe der zweiten Faltungsschicht so um, dass sie mit den vollverknüpften Schichten kompatibel ist. Schließlich wird eine Instanz der Klasse Net erstellt, die das Modell des neuronalen Netzes darstellt.



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
net

In [ ]:
# Install required libraries
#!pip install torchsummary torchviz

# Import required libraries
from torchsummary import summary
from torchviz import make_dot

# Summarize the model
summary(
    net, input_size=(3, 32, 32), device="cpu"
)  # Adjust input_size as per your model's requirement

# Visualize the model architecture
x = torch.randn(1, 3, 32, 32)  # Adjust input size as per your model's requirement
y = net(x)
make_dot(y, params=dict(net.named_parameters())).render("model_architecture_CIFAR", format="png")


In [ ]:
from IPython.display import display
from PIL import Image

# Load and display the image
image = Image.open("model_architecture_CIFAR.png")
display(image)

### Schritt 4: Definieren Sie Verlustfunktion und Optimierer
- In diesem Code wird nn.CrossEntropyLoss() als Verlustfunktion (Kriterium) für das Training des neuronalen Netzes verwendet.
- CrossEntropyLoss" wird üblicherweise für Klassifizierungsaufgaben verwendet und berechnet den Verlust zwischen den vorhergesagten Klassenwahrscheinlichkeiten und den tatsächlichen Klassenbezeichnungen.
- Der Optimierer (`optim.SGD`) wird verwendet, um die Gewichte des neuronalen Netzes während des Trainings zu aktualisieren.
- Als Optimierungsalgorithmus wurde der stochastische Gradientenabstieg (SGD) mit einer Lernrate von 0,001 und einem Impuls von 0,9 gewählt.



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Schritt 5: Trainieren des Netzwerks
- Dieser Code trainiert ein neuronales Netz unter Verwendung des CIFAR-10-Datensatzes mit einer bestimmten Verlustfunktion (Kriterium) und einem Optimierer (Optimizer) für 2 Epochen und druckt den durchschnittlichen Verlust alle 2000 Mini-Batches aus.

In [ ]:
import pytorch_lightning as pl
from torchmetrics.classification import Accuracy
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import CSVLogger
import pandas as pd
import matplotlib.pyplot as plt


class LitModel(pl.LightningModule):
    def __init__(self, model, learning_rate=0.001):
        super().__init__()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

        # Metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.train_acc(outputs, labels)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.val_acc(outputs, labels)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [ ]:
# Define model (replace with your actual model)
net = Net()  # Assuming CNN() is your neural network model

# Use PyTorch Lightning's CSVLogger to store logs
logger = CSVLogger("logs", name="CIFAR_model")

# Define the Lightning Trainer
trainer = pl.Trainer(
    max_epochs=4,  # Train for 4 epochs
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    logger=logger,
    log_every_n_steps=50,
)


In [ ]:
# Train the model
trainer.fit(LitModel(net), trainloader, testloader)

In [ ]:
# ===========================
# 📊 Load and Plot CSV Logs
# ===========================

# Load training logs using pandas
log_file = f"logs/CIFAR_model/version_0/metrics.csv"  # Path to logged metrics
df = pd.read_csv(log_file)

# Aggregate by epoch using `groupby.mean()`
df_grouped = df.groupby("epoch").mean()

df_grouped.head()

In [ ]:
# Plot Training & Validation Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(df_grouped.index, df_grouped["train_loss_epoch"], "o-", label="Train Loss")
plt.plot(df_grouped.index, df_grouped["val_loss"], "s-", label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid()

# Plot Training & Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(df_grouped.index, df_grouped["train_acc_epoch"], "o-", label="Train Accuracy")
plt.plot(df_grouped.index, df_grouped["val_acc"], "s-", label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy")
plt.legend()
plt.grid()

plt.show()


#### Netzwerk speichern und wieder laden

Speichern Sie nach dem Training sowohl die Architektur des Modells als auch seine gelernten Gewichte.
Um das Modell später wiederherzustellen, müssen Sie entweder das gesamte Modell oder sein Zustandswörterbuch laden.


- Option 1 (Speichern des gesamten Modells): Einfach, aber weniger flexibel. Dadurch wird das gespeicherte Modell an den Code und die Umgebung zum Zeitpunkt der Speicherung gebunden.
- Option 2 (Speichern des Zustandswörterbuchs): Ist flexibler und wird häufig verwendet. Sie ermöglicht es Ihnen, nur die Gewichte zu speichern und die Modellstruktur neu zu initialisieren, was bei der Bereitstellung oder gemeinsamen Nutzung von Modellen nützlich ist.





In [ ]:
# Save the entire model (structure and weights)
torch.save(net, "CIFAR_model.pth")

# Alternatively, save only the model's state dictionary (recommended)
torch.save(net.state_dict(), "CIFAR_model_weights.pth")


Option 2: Load the Model Weights (Recommended)
When saving only the state dictionary, you must reinitialize the model structure first:

In [ ]:
# Load the model structure and weights
net = torch.load("CIFAR_model.pth", weights_only=False)
net.eval()  # Set the model to evaluation mode


In [ ]:
# Reinitialize the model structure
net = Net()  # Replace with your model class

# Load the weights
net.load_state_dict(torch.load("model_weights.pth"))
net.eval()  # Set the model to evaluation mode


### Schritt 6: Testen des Netzes
- Dieser Code berechnet die Genauigkeit des neuronalen Netzes auf dem Testdatensatz (Testloader), indem er die vorhergesagten Bezeichnungen mit den tatsächlichen Bezeichnungen vergleicht.
- Er durchläuft den Testdatensatz, berechnet die Ausgaben des Netzes für jedes Bild und vergleicht die vorhergesagten Bezeichnungen mit den tatsächlichen Bezeichnungen.

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))

Die bereitgestellte Ausgabe veranschaulicht den Trainingsprozess eines Convolutional Neural Network (CNN) für den CIFAR-10-Datensatz. Im Laufe von zwei Epochen verringert das Netzwerk schrittweise seine Verlustwerte, was auf eine Verbesserung seiner Fähigkeit, genaue Vorhersagen zu treffen, hinweist. Während des Trainings sinkt der Verlust kontinuierlich von 2,279 auf 1,263, was zeigt, dass das Netzwerk lernt, sich besser an die Trainingsdaten anzupassen. Trotz dieser Verbesserung bleibt die Genauigkeit im Testsatz bei 54 %, was darauf hindeutet, dass das Netzwerk zwar lernt, seine Leistung bei ungesehenen Daten jedoch mäßig ist. Die Verbesserung der Leistung des Modells könnte weitere Experimente mit Hyperparametern, Änderungen der Architektur oder eine Verlängerung der Trainingsdauer beinhalten. Insgesamt zeigt der Trainingsprozess die iterative Natur des Deep Learning, bei dem eine schrittweise Verfeinerung im Laufe der Zeit zu einer verbesserten Leistung führt.



In [ ]:
def plot_predictions(net, dataloader, classes, batch_size=4):
    """
    Plots a batch of images from the given dataloader with predicted labels as titles.

    Args:
        net: The trained neural network model.
        dataloader: PyTorch dataloader (e.g., testloader).
        classes: Tuple of class names in CIFAR-10.
        batch_size: Number of images to display in the batch.
    """
    # Get one batch of images and labels
    data_iter = iter(dataloader)
    images, labels = next(data_iter)

    # Get predictions from the model
    net.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)

    # Create a figure with subplots
    fig, axes = plt.subplots(1, batch_size, figsize=(batch_size * 3, 3))
    for i in range(batch_size):
        img = imshow(images[i])  # Unnormalize and prepare the image
        axes[i].imshow(img)
        axes[i].set_title(f"Pred: {classes[predicted[i]]}\nTrue: {classes[labels[i]]}")
        axes[i].axis("off")  # Hide axes

    plt.tight_layout()  # Adjust layout to avoid overlapping
    plt.show()


# Example usage
plot_predictions(net, testloader, classes, batch_size=4)
